In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gnn-v9-9-2/pytorch/default/1/gnn_model_finish9.pth
/kaggle/input/gnn-v4-2/pytorch/default/1/gnn_model_finish3.pth
/kaggle/input/test-14/sampled_test_all2.parquet
/kaggle/input/trainall5/sampled_train_all_5.parquet
/kaggle/input/trainall3/sampled_train_all_2.parquet
/kaggle/input/gnn-v3-2/pytorch/default/1/gnn_model_finish2.pth
/kaggle/input/gnn/pytorch/default/1/gnn_model_finish.pth
/kaggle/input/gnn-v7-2/pytorch/default/1/gnn_model_finish7.pth
/kaggle/input/gnn-v3/pytorch/default/1/gnn_model_finish_2.pth
/kaggle/input/trainall6/sampled_train_all_6.parquet
/kaggle/input/test-110/sampled_test_all.parquet
/kaggle/input/gnn-v8-2/pytorch/default/1/gnn_model_finish8-2.pth
/kaggle/input/gnn-/pytorch/default/1/gnn_model_finish_4.pth
/kaggle/input/trainall4/sampled_train_all_4.parquet
/kaggle/input/trainall2/sampled_train_all_3.parquet
/kaggle/input/gnn-v2/pytorch/default/1/gnn_model_finish.pth
/kaggle/input/trainall7/sampled_train_all_7.parquet
/kaggle/input/trainall1/sampled_tr

In [3]:
# 1. 安裝 PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


# 2. 安裝 RDKit（用於處理分子）
!pip install rdkit-pypi


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 25.6 MB/s eta 0:00:0000:01:00:01


In [4]:
%%writefile requirements.txt
easydict
future
matplotlib
numpy
opencv-python
scikit-image
scipy
click
requests
tqdm
pyspng
ninja
imageio-ffmpeg==0.4.3
timm
psutil
scikit-learn

Writing requirements.txt


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 5.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.5.1
    Uninstalling imageio-ffmpeg-0.5.1:
      Successfully uninstalled imageio-ffmpeg-0.5.1


In [6]:
# 1. 升級 PyTorch 和 torchvision
#!pip install torch==2.1.0 torchvision==0.15.2 torchaudio==2.0.2 torchdata==0.6.1
!pip install torch torchvision torchaudio torchdata --index-url https://download.pytorch.org/whl/cu118
!pip install pytorch-lightning


# 3. 檢查安裝是否成功
import torch
import torchdata


print("Torch version:", torch.__version__)
print("Torchdata version:", torchdata.__version__)




Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.7 MB/s eta 0:00:00
Torch version: 2.5.1+cu121
Torchdata version: 0.10.0


In [7]:
!python --version
!nvcc --version  # 如果已經安裝了CUDA


Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [8]:
!pip install duckdb

In [9]:
!pip install pyarrow  # 或者你也可以選擇 fastparquet


In [10]:

import pyarrow.parquet as pq
import pandas as pd
import gc  # 引入垃圾回收模組

filename = "/kaggle/input/leash-BELKA/train.parquet"
columns_to_read = ["molecule_smiles", "protein_name", "binds"]

batch_size = 60000    # 每次讀取 60,000 筆
target_rows = 1200000  # 每輪儲存 1,200,000 筆
total_batches = 15    # 總共執行 15 次
total_rows = 0        # 記錄當前累積筆數

parquet_file = pq.ParquetFile(filename)

# 獲取總行組數量
num_row_groups = parquet_file.num_row_groups
print(f"Total row groups in file: {num_row_groups}")

# 計算每次讀取多少行組
row_groups_per_batch = target_rows // batch_size


# 開始進行批次處理
for i in range(total_batches):
    chunks = []
    current_rows = 0  # 每輪的計數器
    batch_start_row_group = i * row_groups_per_batch  # 直接依序取
    batch_end_row_group = min(batch_start_row_group + row_groups_per_batch, num_row_groups)

    if batch_end_row_group >= num_row_groups:
        batch_end_row_group = num_row_groups  # 避免超出行組範圍

    print(f"✅ 第 {i+1} 次處理：從行組 {batch_start_row_group} 到行組 {batch_end_row_group}")

    # 使用 pyarrow 的 ParquetFile 直接讀取指定範圍的行組
    for row_group_idx in range(batch_start_row_group, batch_end_row_group):
        try:
            batch = parquet_file.read_row_groups([row_group_idx], columns=columns_to_read)
            chunk = batch.to_pandas()

            # 檢查是否有資料
            if not chunk.empty:
                chunks.append(chunk)
                current_rows += len(chunk)
                total_rows += len(chunk)

            # 如果讀取到指定範圍的資料，就停止
            if total_rows >= target_rows * (i + 1):
                break  # 如果已經讀到該批次的結尾就停止

        except Exception as e:
            print(f"⚠️ 讀取行組 {row_group_idx} 時發生錯誤: {e}")

    if chunks:  # 確保有資料才進行合併
        # 合併 DataFrame
        batch_df = pd.concat(chunks, ignore_index=True)

        # **將每 3 列合併成 1 列**
        batch_df["row_idx"] = batch_df.index // 3  # 每 3 列分組
        batch_pivot = batch_df.pivot(index="row_idx", columns="protein_name", values="binds").reset_index()

        # **合併 molecule_smiles**
        smiles_df = batch_df.groupby("row_idx")["molecule_smiles"].first().reset_index()
        final_df = smiles_df.merge(batch_pivot, on="row_idx").drop(columns=["row_idx"])

        # 存成 parquet，每次都存不同的檔案
        output_filename = f"/kaggle/working/train_part{i+1}.parquet"
        final_df.to_parquet(output_filename, index=False)

        print(f"✅ 第 {i+1} 次存檔：{len(final_df)} 筆，已累積 {total_rows} 筆")

        # 清理無用的變數，釋放記憶體
        del batch_df, batch_pivot, smiles_df, final_df
        gc.collect()  # 執行垃圾回收
    else:
        print(f"⚠️ 第 {i+1} 次處理未讀取到任何資料，跳過該批次。")


'\nimport pyarrow.parquet as pq\nimport pandas as pd\nimport gc  # 引入垃圾回收模組\n\nfilename = "/kaggle/input/leash-BELKA/train.parquet"\ncolumns_to_read = ["molecule_smiles", "protein_name", "binds"]\n\nbatch_size = 60000    # 每次讀取 60,000 筆\ntarget_rows = 1200000  # 每輪儲存 1,200,000 筆\ntotal_batches = 15    # 總共執行 15 次\ntotal_rows = 0        # 記錄當前累積筆數\n\nparquet_file = pq.ParquetFile(filename)\n\n# 獲取總行組數量\nnum_row_groups = parquet_file.num_row_groups\nprint(f"Total row groups in file: {num_row_groups}")\n\n# 計算每次讀取多少行組\nrow_groups_per_batch = target_rows // batch_size\n\n\n# 開始進行批次處理\nfor i in range(total_batches):\n    chunks = []\n    current_rows = 0  # 每輪的計數器\n    batch_start_row_group = i * row_groups_per_batch  # 直接依序取\n    batch_end_row_group = min(batch_start_row_group + row_groups_per_batch, num_row_groups)\n\n    if batch_end_row_group >= num_row_groups:\n        batch_end_row_group = num_row_groups  # 避免超出行組範圍\n\n    print(f"✅ 第 {i+1} 次處理：從行組 {batch_start_row_group} 到行組 {batch_e

In [11]:
# 15 個 Parquet 檔案
parquet_files = [f"/kaggle/working/train_part{i+1}.parquet" for i in range(0, 15)]

# 初始化 DuckDB 連線
con = duckdb.connect()

# 存放所有批次的 DataFrame
all_samples = []

# 逐個處理 15 個檔案
for i, file in enumerate(parquet_files):
    print(f"📂 正在處理檔案: {file}")

    df = con.query(f"""(SELECT * FROM parquet_scan('{file}')
                            WHERE BRD4 = 0 and HSA = 0 and sEH = 0
                            ORDER BY random()
                            LIMIT 3000)
                            UNION ALL
                            (SELECT * FROM parquet_scan('{file}')
                            WHERE BRD4 = 1 or HSA = 1 or sEH = 1
                            ORDER BY random()
                            LIMIT 13000)""").df()

# 儲存該批次結果
    output_filename = f"/kaggle/working/sampled_test_part{i+1}.parquet"
    df.to_parquet(output_filename, index=False)
    print(f"✅ 已儲存抽樣結果: {output_filename}（共 {len(df)} 筆）")

    all_samples.append(df)

# 合併所有結果
final_test_df = pd.concat(all_samples, ignore_index=True)

# 儲存總合併的 Parquet
final_test_output = "/kaggle/working/sampled_test_all.parquet"
final_test_df.to_parquet(final_test_output, index=False)
print(f"🎯 全部 15 個檔案已處理完畢，最終合併檔案: {final_test_output}（共 {len(final_test_df)} 筆）")

# 關閉 DuckDB
con.close()

'\n\n'

In [12]:
'''
# 15 個 Parquet 檔案
parquet_files = [f"/kaggle/working/train_part{i+1}.parquet" for i in range(0, 15)]

# 初始化 DuckDB 連線
con = duckdb.connect()

# 存放所有批次的 DataFrame
all_samples = []

# 逐個處理 15 個檔案
for i, file in enumerate(parquet_files):
    print(f"📂 正在處理檔案: {file}")

    df = con.query(f"""(SELECT * FROM parquet_scan('{file}')
                            ORDER BY random()
                            LIMIT 15000)
                            UNION ALL
                            (SELECT * FROM parquet_scan('{file}')
                            WHERE BRD4 = 1 or HSA = 1 or sEH = 1
                            ORDER BY random()
                            LIMIT 5000)""").df()

# 儲存該批次結果
    output_filename = f"/kaggle/working/sampled_test_part{i+1}.parquet"
    df.to_parquet(output_filename, index=False)
    print(f"✅ 已儲存抽樣結果: {output_filename}（共 {len(df)} 筆）")

    all_samples.append(df)

# 合併所有結果
final_test_df = pd.concat(all_samples, ignore_index=True)

# 儲存總合併的 Parquet
final_test_output = "/kaggle/working/sampled_test_all.parquet"
final_test_df.to_parquet(final_test_output, index=False)
print(f"🎯 全部 15 個檔案已處理完畢，最終合併檔案: {final_test_output}（共 {len(final_test_df)} 筆）")

# 關閉 DuckDB
con.close()
'''
#y = df[["BRD4", "HSA", "sEH"]].values

'\n# 15 個 Parquet 檔案\nparquet_files = [f"/kaggle/working/train_part{i+1}.parquet" for i in range(0, 15)]\n\n# 初始化 DuckDB 連線\ncon = duckdb.connect()\n\n# 存放所有批次的 DataFrame\nall_samples = []\n\n# 逐個處理 15 個檔案\nfor i, file in enumerate(parquet_files):\n    print(f"📂 正在處理檔案: {file}")\n\n    df = con.query(f"""(SELECT * FROM parquet_scan(\'{file}\')\n                            ORDER BY random()\n                            LIMIT 15000)\n                            UNION ALL\n                            (SELECT * FROM parquet_scan(\'{file}\')\n                            WHERE BRD4 = 1 or HSA = 1 or sEH = 1\n                            ORDER BY random()\n                            LIMIT 5000)""").df()\n\n# 儲存該批次結果\n    output_filename = f"/kaggle/working/sampled_test_part{i+1}.parquet"\n    df.to_parquet(output_filename, index=False)\n    print(f"✅ 已儲存抽樣結果: {output_filename}（共 {len(df)} 筆）")\n\n    all_samples.append(df)\n\n# 合併所有結果\nfinal_test_df = pd.concat(all_samples, ignore_index=True)

In [13]:
# df.head()

In [14]:

output_filename = f"/kaggle/working/training_df.csv"
df.to_csv(output_filename, index=False)


'\noutput_filename = f"/kaggle/working/training_df.csv"\ndf.to_csv(output_filename, index=False)\n'

In [15]:
!pip install torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.1+cu118.html

!pip install torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.0+${CUDA}.html

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 4.4 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 3.4 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.7/947.7 kB 1.1 MB/s eta 0:00:0000:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.0+.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3879861 sha256=118b2ed2c180951d1316cfa8fa1a306c9049665a76b3fac4923a01aa771ab8ab
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter


In [16]:
import torch
print(torch.cuda.is_available())  # True
print(torch.version.cuda)  # 應該顯示 11.8


True
12.1


In [17]:
import torch
import torch_scatter
print("torch-scatter 安裝成功！")

torch-scatter 安裝成功！


In [18]:
import numpy as np

import rdkit
from rdkit import Chem

import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_scatter import scatter

import torch
import torch.nn as nn
import torch.nn.functional as F

print('import ok!')

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


import ok!


In [19]:
# helper
# torch version of np unpackbits
#https://gist.github.com/vadimkantorov/30ea6d278bc492abf6ad328c6965613a

def tensor_dim_slice(tensor, dim, dim_slice):
	return tensor[(dim if dim >= 0 else dim + tensor.dim()) * (slice(None),) + (dim_slice,)]

# @torch.jit.script
def packshape(shape, dim: int = -1, mask: int = 0b00000001, dtype=torch.uint8, pack=True):
	dim = dim if dim >= 0 else dim + len(shape)
	bits, nibble = (
		8 if dtype is torch.uint8 else 16 if dtype is torch.int16 else 32 if dtype is torch.int32 else 64 if dtype is torch.int64 else 0), (
		1 if mask == 0b00000001 else 2 if mask == 0b00000011 else 4 if mask == 0b00001111 else 8 if mask == 0b11111111 else 0)
	# bits = torch.iinfo(dtype).bits # does not JIT compile
	assert nibble <= bits and bits % nibble == 0
	nibbles = bits // nibble
	shape = (shape[:dim] + (int(math.ceil(shape[dim] / nibbles)),) + shape[1 + dim:]) if pack else (
				shape[:dim] + (shape[dim] * nibbles,) + shape[1 + dim:])
	return shape, nibbles, nibble

# @torch.jit.script
def F_unpackbits(tensor, dim: int = -1, mask: int = 0b00000001, shape=None, out=None, dtype=torch.uint8):
	dim = dim if dim >= 0 else dim + tensor.dim()
	shape_, nibbles, nibble = packshape(tensor.shape, dim=dim, mask=mask, dtype=tensor.dtype, pack=False)
	shape = shape if shape is not None else shape_
	out = out if out is not None else torch.empty(shape, device=tensor.device, dtype=dtype)
	assert out.shape == shape

	if shape[dim] % nibbles == 0:
		shift = torch.arange((nibbles - 1) * nibble, -1, -nibble, dtype=torch.uint8, device=tensor.device)
		shift = shift.view(nibbles, *((1,) * (tensor.dim() - dim - 1)))
		return torch.bitwise_and((tensor.unsqueeze(1 + dim) >> shift).view_as(out), mask, out=out)

	else:
		for i in range(nibbles):
			shift = nibble * i
			sliced_output = tensor_dim_slice(out, dim, slice(i, None, nibbles))
			sliced_input = tensor.narrow(dim, 0, sliced_output.shape[dim])
			torch.bitwise_and(sliced_input >> shift, mask, out=sliced_output)
	return out

class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__
	
	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

            
print('helper ok!')

helper ok!


In [20]:
# mol to graph adopted from
# from https://github.com/LiZhang30/GPCNDTA/blob/main/utils/DrugGraph.py

PACK_NODE_DIM=9
PACK_EDGE_DIM=1
NODE_DIM=PACK_NODE_DIM*8
EDGE_DIM=PACK_EDGE_DIM*8

def one_of_k_encoding(x, allowable_set, allow_unk=False):
	if x not in allowable_set:
		if allow_unk:
			x = allowable_set[-1]
		else:
			raise Exception(f'input {x} not in allowable set{allowable_set}!!!')
	return list(map(lambda s: x == s, allowable_set))


#Get features of an atom (one-hot encoding:)
'''
	1.atom element: 44+1 dimensions    
	2.the atom's hybridization: 5 dimensions
	3.degree of atom: 6 dimensions                        
	4.total number of H bound to atom: 6 dimensions
	5.number of implicit H bound to atom: 6 dimensions    
	6.whether the atom is on ring: 1 dimension
	7.whether the atom is aromatic: 1 dimension           
	Total: 70 dimensions
'''

ATOM_SYMBOL = [
	'C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg',
	'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl',
	'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H',
	'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
	'Pt', 'Hg', 'Pb', 'Dy',
	#'Unknown'
]
#print('ATOM_SYMBOL', len(ATOM_SYMBOL))44
HYBRIDIZATION_TYPE = [
	Chem.rdchem.HybridizationType.S,
	Chem.rdchem.HybridizationType.SP,
	Chem.rdchem.HybridizationType.SP2,
	Chem.rdchem.HybridizationType.SP3,
	Chem.rdchem.HybridizationType.SP3D
]

def get_atom_feature(atom):
	feature = (
		 one_of_k_encoding(atom.GetSymbol(), ATOM_SYMBOL)
	   + one_of_k_encoding(atom.GetHybridization(), HYBRIDIZATION_TYPE)
	   + one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5])
	   + one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5])
	   + one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5])
	   + [atom.IsInRing()]
	   + [atom.GetIsAromatic()]
	)
	#feature = np.array(feature, dtype=np.uint8)
	feature = np.packbits(feature)
	return feature


#Get features of an edge (one-hot encoding)
'''
	1.single/double/triple/aromatic: 4 dimensions       
	2.the atom's hybridization: 1 dimensions
	3.whether the bond is on ring: 1 dimension          
	Total: 6 dimensions
'''

def get_bond_feature(bond):
	bond_type = bond.GetBondType()
	feature = [
		bond_type == Chem.rdchem.BondType.SINGLE,
		bond_type == Chem.rdchem.BondType.DOUBLE,
		bond_type == Chem.rdchem.BondType.TRIPLE,
		bond_type == Chem.rdchem.BondType.AROMATIC,
		bond.GetIsConjugated(),
		bond.IsInRing()
	]
	#feature = np.array(feature, dtype=np.uint8)
	feature = np.packbits(feature)
	return feature


def smile_to_graph(smiles):
	mol = Chem.MolFromSmiles(smiles)
	N = mol.GetNumAtoms()
	node_feature = []
	edge_feature = []
	edge = []
	for i in range(mol.GetNumAtoms()):
		atom_i = mol.GetAtomWithIdx(i)
		atom_i_features = get_atom_feature(atom_i)
		node_feature.append(atom_i_features)

		for j in range(mol.GetNumAtoms()):
			bond_ij = mol.GetBondBetweenAtoms(i, j)
			if bond_ij is not None:
				edge.append([i, j])
				bond_features_ij = get_bond_feature(bond_ij)
				edge_feature.append(bond_features_ij)
	node_feature=np.stack(node_feature)
	edge_feature=np.stack(edge_feature)
	edge = np.array(edge,dtype=np.uint8)
	return N,edge,node_feature,edge_feature

def to_pyg_format(N,edge,node_feature,edge_feature):
	graph = Data(
		idx=-1,
		edge_index = torch.from_numpy(edge.T).int(),
		x          = torch.from_numpy(node_feature).byte(),
		edge_attr  = torch.from_numpy(edge_feature).byte(),
	)
	return graph

#debug one example
g = to_pyg_format(*smile_to_graph(smiles="C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1"))
print(g)
print('[Dy] is replaced by C !!')
print('smile_to_graph() ok!')

Data(x=[37, 9], edge_index=[2, 78], edge_attr=[78, 1], idx=-1)
[Dy] is replaced by C !!
smile_to_graph() ok!


In [21]:
#MODEL: simple MPNNModel
#from https://github.com/chaitjo/geometric-gnn-dojo/blob/main/geometric_gnn_101.ipynb

#DEVICE='cuda'
DEVICE='cpu'

# i have removed all comments here to jepp it clean. refer to orginal link for code comments
# of MPNNModel
class MPNNLayer(MessagePassing):
    def __init__(self, emb_dim=64, edge_dim=4, aggr='add'):
        super().__init__(aggr=aggr)
    
        self.emb_dim = emb_dim
        self.edge_dim = edge_dim
        self.mlp_msg = nn.Sequential(
            nn.Linear(2 * emb_dim + edge_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU(),
            nn.Linear(emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU()
        )
        self.mlp_upd = nn.Sequential(
            nn.Linear(2 * emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU(),
            nn.Linear(emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU()
        )
    
    def forward(self, h, edge_index, edge_attr):
        out = self.propagate(edge_index, h=h, edge_attr=edge_attr)
        return out
    
    def message(self, h_i, h_j, edge_attr):
        msg = torch.cat([h_i, h_j, edge_attr], dim=-1)
        return self.mlp_msg(msg)
    
    def aggregate(self, inputs, index):
        return scatter(inputs, index, dim=self.node_dim, reduce=self.aggr)
    
    def update(self, aggr_out, h):
        upd_out = torch.cat([h, aggr_out], dim=-1)
        return self.mlp_upd(upd_out)
    
    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(emb_dim={self.emb_dim}, aggr={self.aggr})')
    

class MPNNModel(nn.Module):
    def __init__(self, num_layers=4, emb_dim=64, in_dim=11, edge_dim=4, out_dim=1):
        super().__init__()
    
        self.lin_in = nn.Linear(in_dim, emb_dim)
    
        # Stack of MPNN layers
        self.convs = torch.nn.ModuleList()
        for layer in range(num_layers):
            self.convs.append(MPNNLayer(emb_dim, edge_dim, aggr='add'))
    
        self.pool = global_mean_pool
    
    def forward(self, data): #PyG.Data - batch of PyG graphs
    
        h = self.lin_in(F_unpackbits(data.x,-1).float())  
    
        for conv in self.convs:
            h = h + conv(h, data.edge_index.long(), F_unpackbits(data.edge_attr,-1).float())  # (n, d) -> (n, d)
    
        h_graph = self.pool(h, data.batch)  
        return h_graph

# our prediction model here !!!!
class Net(nn.Module):
    def __init__(self, ):
        super().__init__()
    
        self.output_type = ['infer', 'loss']
    
        graph_dim=96
        self.smile_encoder = MPNNModel(
             in_dim=NODE_DIM, edge_dim=EDGE_DIM, emb_dim=graph_dim, num_layers=4,
        )
        self.bind = nn.Sequential(
            nn.Linear(graph_dim, 1024),
            #nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(1024, 1024),
            #nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 3),
        )
    
    def forward(self, batch):
        graph = batch['graph']
        x = self.smile_encoder(graph) 
        bind = self.bind(x)
    
        # --------------------------
        output = {}
        if 'loss' in self.output_type:
            target = batch['bind']
            output['bce_loss'] = F.binary_cross_entropy_with_logits(bind.float(), target.float())
            """
            # Debugging loss calculation新增
            print("Bind Output (Sigmoid):", torch.sigmoid(bind).detach().cpu().numpy())
            print("Target:", target.cpu().numpy())
            print("Loss Value:", output['bce_loss'].item())
            """
    
        if 'infer' in self.output_type:
            output['bind'] = torch.sigmoid(bind)
    
        return output
    
#debug: make some dummy data and run
'''
def run_check_net():
    batch_size = 30
    node_dim=NODE_DIM
    edge_dim=EDGE_DIM
    
    data = []
    for b in range(batch_size):
        N = np.random.randint(5,10)
        E = np.random.randint(3,N*(N-1))
        edge_index = np.stack([
            np.random.choice(N, E, replace=True),
            np.random.choice(N, E, replace=True),
        ]).T
        edge_index = np.sort(edge_index)
        edge_index = edge_index[edge_index[:, 0].argsort()]
        edge_index[0] = [0,1] #default
        edge_index = edge_index[edge_index[:,0]!=edge_index[:,1]]
        edge_index = np.unique(edge_index, axis=0)
    
        E = len(edge_index)
        edge_index = np.ascontiguousarray(edge_index.T)
    
        d = Data(
            idx        = b,
            edge_index = torch.from_numpy(edge_index).int(),
            x          = torch.from_numpy(np.packbits(np.random.choice(2, (N, node_dim)),-1)).byte(),
            edge_attr  = torch.from_numpy(np.packbits(np.random.choice(2, (E, edge_dim)),-1)).byte(),
        )
        data.append(d)
    
    
    loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    epoch_indices = []  # 儲存 index 以檢查跳變
    for batch in loader:
        epoch_indices.extend(batch.idx.tolist())
    
    
    # 📌 記錄當前 epoch 所有 batch index
    print(f"Epoch index range: {min(epoch_indices)} → {max(epoch_indices)}")
    
    
    # loader = DataLoader(data, batch_size=batch_size)
    graph = next(iter(loader))
    idx = graph.idx.tolist()  #use to index bind array
    batch = dotdict( 
        graph = graph.to(DEVICE),
        bind  = torch.from_numpy(np.random.choice(2, (batch_size, 3))).float().to(DEVICE),
    )
    zz=0
    
    net = Net().to(DEVICE)
    #print(net)
    
    with torch.no_grad():
        with torch.amp.autocast('cuda', enabled=True): # dtype=torch.float16):
            output = net(batch)
            #print(output['bind'])
    
    # ---
    print('batch')
    for k, v in batch.items():
        if k=='idx':
            print(f'{k:>32} : {len(v)} ')
        elif k=='graph':
            print(f'{k:>32} : {graph} ')
        else:
            print(f'{k:>32} : {v.shape} ')
    
    print('output')
    for k, v in output.items():
        if 'loss' not in k:
            print(f'{k:>32} : {v.shape} ')
    print('loss')
    for k, v in output.items():
        if 'loss' in k:
            print(f'{k:>32} : {v.item()} ')
    
                
run_check_net()
'''
print('model ok!')


model ok!


In [22]:
#example of parallel conversion of smiles to graph
'''
from multiprocessing import Pool
from tqdm import tqdm
import gc
from torch_geometric.loader import DataLoader as PyGDataLoader

def to_pyg_list(graph):
	L = len(graph)
	for i in tqdm(range(L)):
		N, edge, node_feature, edge_feature = graph[i]
		graph[i] = Data(
			idx=i,
			edge_index=torch.from_numpy(edge.T).int(),
			x=torch.from_numpy(node_feature).byte(),
			edge_attr=torch.from_numpy(edge_feature).byte(),
		)
	return graph


train_smiles=[ #replace [Dy] with C
    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
]
train_bind =np.array([
    [0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,0,0],
])
num_train= len(train_smiles)
with Pool(processes=64) as pool:
    train_graph = list(tqdm(pool.imap(smile_to_graph, train_smiles), total=num_train))

train_graph = to_pyg_list(train_graph)
train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=True)

## example training loop
scaler = torch.cuda.amp.GradScaler(enabled=True)
net = Net()
net.to(DEVICE)

optimizer =\
	torch.optim.AdamW(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001)

num_epoch=10
epoch=0
iteration=0
while epoch<num_epoch: 
	for t, graph_batch in enumerate(train_loader): 
		index = graph_batch.idx.tolist()
		B = len(index)
		batch = dotdict(
			graph  = graph_batch.to(DEVICE),
			bind   = torch.from_numpy(train_bind[index]).to(DEVICE),
		)

		net.train()
		net.output_type = ['loss', 'infer']
		with torch.cuda.amp.autocast(enabled=True):
			output = net(batch)  #data_parallel(net,batch) #
			bce_loss = output['bce_loss']

		optimizer.zero_grad() 
		scaler.scale(bce_loss).backward() 
		scaler.step(optimizer)
		scaler.update()
		 
		torch.clear_autocast_cache()
		print(epoch,iteration,bce_loss.item())
		iteration +=  1
        
	epoch += 1
'''

'\nfrom multiprocessing import Pool\nfrom tqdm import tqdm\nimport gc\nfrom torch_geometric.loader import DataLoader as PyGDataLoader\n\ndef to_pyg_list(graph):\n\tL = len(graph)\n\tfor i in tqdm(range(L)):\n\t\tN, edge, node_feature, edge_feature = graph[i]\n\t\tgraph[i] = Data(\n\t\t\tidx=i,\n\t\t\tedge_index=torch.from_numpy(edge.T).int(),\n\t\t\tx=torch.from_numpy(node_feature).byte(),\n\t\t\tedge_attr=torch.from_numpy(edge_feature).byte(),\n\t\t)\n\treturn graph\n\n\ntrain_smiles=[ #replace [Dy] with C\n    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",\n    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",\n    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",\n    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",\n    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",\n    "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",\n]\ntrain_bind =np.array([\n    [0,0,0],[1,0,0],[

In [23]:
'''
import numpy as np
import torch
from torch_geometric.data import Data
from multiprocessing import Pool
from tqdm import tqdm
from torch_geometric.loader import DataLoader as PyGDataLoader


# 轉換為 PyG 格式
def to_pyg_list(graph):
	L = len(graph)
	for i in tqdm(range(L)):
		N, edge, node_feature, edge_feature = graph[i]
		graph[i] = Data(
			idx=i,
			edge_index=torch.from_numpy(edge.T).int(),
			x=torch.from_numpy(node_feature).byte(),
			edge_attr=torch.from_numpy(edge_feature).byte(),
		)
	return graph


train_file = "/kaggle/input/trainall0/sampled_train_all.parquet"
df = pd.read_parquet(train_file)


# train_loader
# 從 df 讀取 SMILES
train_smiles = df['molecule_smiles'].tolist()


train_bind = df[["BRD4", "HSA", "sEH"]].values
#train_bind = np.array([[1, 0, 0] if b == 1 else [0, 0, 0] for b in train_bind])  # 假設 binds 是 0/1

num_train = len(train_smiles)

# 平行處理 SMILES 轉 Graph
with Pool(processes=8) as pool:  # 設為 8 核心，避免記憶體爆炸
    train_graph = list(tqdm(pool.imap(smile_to_graph, train_smiles), total=num_train))


train_graph = to_pyg_list(train_graph)
#train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=True)
train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=True, worker_init_fn=np.random.seed(42))



# val_loader

val_file = "/kaggle/working/sampled_train_all.parquet"
val_df = pd.read_parquet(val_file)

# 從 df 讀取 SMILES
val_smiles = val_df['molecule_smiles'].tolist()


val_bind = val_df[["BRD4", "HSA", "sEH"]].values


num_val = len(val_smiles)

# 平行處理 SMILES 轉 Graph
with Pool(processes=8) as pool:  # 設為 8 核心，避免記憶體爆炸
    val_graph = list(tqdm(pool.imap(smile_to_graph, val_smiles), total=num_val))


val_graph = to_pyg_list(val_graph)

num_val_samples = 20000  # 取 20,000 筆
total_val_samples = len(val_graph)

# 隨機選取 20,000 個索引（如果數據少於 20,000，則全部選取）
random_indices = np.random.choice(total_val_samples, min(num_val_samples, total_val_samples), replace=False)

# 選取對應的 Graph 和 Binding Data
val_graph_subset = [val_graph[i] for i in random_indices]
val_bind_subset = val_bind[random_indices] 
val_bind =val_bind_subset


# 建立 DataLoader
val_loader = PyGDataLoader(val_graph_subset, batch_size=3, shuffle=True)



## example training loop
scaler = torch.cuda.amp.GradScaler(enabled=True)
#net = Net()
net = torch.load("/kaggle/input/gnn/pytorch/default/1/gnn_model_finish.pth")
net.to(DEVICE)

optimizer =\
	torch.optim.AdamW(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001)

num_epoch=5
epoch=0
iteration=0


while epoch<num_epoch: 
    for t, graph_batch in enumerate(train_loader): 
        index = graph_batch.idx.tolist()
        B = len(index)
        batch = dotdict(
			graph  = graph_batch.to(DEVICE),
			bind   = torch.from_numpy(train_bind[index]).to(DEVICE),
		)

        net.train()
        net.output_type = ['loss', 'infer']
        with torch.cuda.amp.autocast(enabled=True):
            output = net(batch)  #data_parallel(net,batch) 
            bce_loss = output['bce_loss']

        optimizer.zero_grad() 
        scaler.scale(bce_loss).backward() 
        scaler.step(optimizer)
        scaler.update()
		 
        torch.clear_autocast_cache()
        print(epoch,iteration,bce_loss.item())
        iteration +=  1

    # ===== Validation Step =====
    net.eval()  # 切換為評估模式
    val_loss = 0.0
    num_batches = 0
    num_samples = 0
    
    with torch.no_grad():
        for batch_idx, val_graph_batch in enumerate(val_loader):  # 使用 enumerate 確保 batch 索引
            if num_samples >= num_val_samples:
                break  # 達到 50,000 筆後結束驗證
    
            # 這裡不能用 val_graph_batch.idx.tolist()，改成直接取 batch_idx
            val_batch = dotdict(
                graph = val_graph_batch.to(DEVICE),
                bind = torch.from_numpy(val_bind_subset[batch_idx * 3 : (batch_idx + 1) * 3]).to(DEVICE),
            )
    
            net.output_type = ['loss', 'infer']
            val_output = net(val_batch)
            val_loss += val_output['bce_loss'].item()
            num_batches += 1
            num_samples += len(val_batch.graph)
    
    avg_val_loss = val_loss / num_batches
    print(f"Validation Loss: {avg_val_loss:.4f}")

    epoch += 1

'''

'\nimport numpy as np\nimport torch\nfrom torch_geometric.data import Data\nfrom multiprocessing import Pool\nfrom tqdm import tqdm\nfrom torch_geometric.loader import DataLoader as PyGDataLoader\n\n\n# 轉換為 PyG 格式\ndef to_pyg_list(graph):\n\tL = len(graph)\n\tfor i in tqdm(range(L)):\n\t\tN, edge, node_feature, edge_feature = graph[i]\n\t\tgraph[i] = Data(\n\t\t\tidx=i,\n\t\t\tedge_index=torch.from_numpy(edge.T).int(),\n\t\t\tx=torch.from_numpy(node_feature).byte(),\n\t\t\tedge_attr=torch.from_numpy(edge_feature).byte(),\n\t\t)\n\treturn graph\n\n\ntrain_file = "/kaggle/input/trainall0/sampled_train_all.parquet"\ndf = pd.read_parquet(train_file)\n\n\n# train_loader\n# 從 df 讀取 SMILES\ntrain_smiles = df[\'molecule_smiles\'].tolist()\n\n\ntrain_bind = df[["BRD4", "HSA", "sEH"]].values\n#train_bind = np.array([[1, 0, 0] if b == 1 else [0, 0, 0] for b in train_bind])  # 假設 binds 是 0/1\n\nnum_train = len(train_smiles)\n\n# 平行處理 SMILES 轉 Graph\nwith Pool(processes=8) as pool:  # 設為 8 核心，避免記憶體

In [24]:

import numpy as np
import torch
from torch_geometric.data import Data
from multiprocessing import Pool
from tqdm import tqdm
from torch_geometric.loader import DataLoader as PyGDataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 轉換為 PyG 格式
def to_pyg_list(graph):
	L = len(graph)
	for i in tqdm(range(L)):
		N, edge, node_feature, edge_feature = graph[i]
		graph[i] = Data(
			idx=i,
			edge_index=torch.from_numpy(edge.T).int(),
			x=torch.from_numpy(node_feature).byte(),
			edge_attr=torch.from_numpy(edge_feature).byte(),
		)
	return graph


train_file = '/kaggle/input/trainall7/sampled_train_all_7.parquet'
df = pd.read_parquet(train_file)


# train_loader
# 從 df 讀取 SMILES
train_smiles = df['molecule_smiles'].tolist()


train_bind = df[["BRD4", "HSA", "sEH"]].values
#train_bind = np.array([[1, 0, 0] if b == 1 else [0, 0, 0] for b in train_bind])  # 假設 binds 是 0/1

num_train = len(train_smiles)

# 平行處理 SMILES 轉 Graph
with Pool(processes=8) as pool:  # 設為 8 核心，避免記憶體爆炸
    train_graph = list(tqdm(pool.imap(smile_to_graph, train_smiles), total=num_train))


train_graph = to_pyg_list(train_graph)
#train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=True)
#train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=True, worker_init_fn=np.random.seed(42))
train_loader = PyGDataLoader(train_graph, batch_size=9, shuffle=True, drop_last=False, worker_init_fn=np.random.seed(42))


# val_loader

val_file = "/kaggle/input/trainall1/sampled_train_all_1.parquet"
val_df = pd.read_parquet(val_file)

# 從 df 讀取 SMILES
val_smiles = val_df['molecule_smiles'].tolist()


val_bind = val_df[["BRD4", "HSA", "sEH"]].values


num_val = len(val_smiles)

# 平行處理 SMILES 轉 Graph
with Pool(processes=8) as pool:  # 設為 8 核心，避免記憶體爆炸
    val_graph = list(tqdm(pool.imap(smile_to_graph, val_smiles), total=num_val))


val_graph = to_pyg_list(val_graph)

num_val_samples = 20000  # 取 20,000 筆
total_val_samples = len(val_graph)

# 隨機選取 20,000 個索引（如果數據少於 20,000，則全部選取）
random_indices = np.random.choice(total_val_samples, min(num_val_samples, total_val_samples), replace=False)

# 選取對應的 Graph 和 Binding Data
val_graph_subset = [val_graph[i] for i in random_indices]
val_bind_subset = val_bind[random_indices] 
val_bind =val_bind_subset


# 建立 DataLoader
val_loader = PyGDataLoader(val_graph_subset, batch_size=9, shuffle=False, drop_last=False)




## example training loop
scaler = torch.cuda.amp.GradScaler(enabled=True)
# scaler = torch.amp.GradScaler(enabled=True)

#net = Net()
net = torch.load("/kaggle/input/gnn-v8-2/pytorch/default/1/gnn_model_finish8-2.pth")
net.to(DEVICE)
#net = torch.load("/kaggle/input/gnn-/pytorch/default/1/gnn_model_finish_4.pth", map_location=torch.device('cpu'))


optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, net.parameters()), lr=0.0004)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3)

num_epoch=9
epoch=0
iteration=0


while epoch < num_epoch:
    
    epoch_loss = 0.0  # 紀錄整個 epoch 的 loss

    for t, graph_batch in enumerate(train_loader): 
        epoch_indices = []  # 儲存 index 以檢查跳變
        index = graph_batch.idx.tolist()
        epoch_indices.extend(index)
        B = len(index)

        batch = dotdict(
            graph = graph_batch.to(DEVICE),
            bind = torch.from_numpy(train_bind[index]).to(DEVICE),
        )

        # 📌 記錄當前 epoch 所有 batch index
        print(f"Epoch {epoch+1}, Batch {t+1} - Index range: {min(epoch_indices)} → {max(epoch_indices)}")

        net.train()
        net.output_type = ['loss', 'infer']
        
        with torch.amp.autocast(device_type='cuda', enabled=True):
            output = net(batch)
            bce_loss = output['bce_loss']

        optimizer.zero_grad()
        scaler.scale(bce_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        torch.cuda.empty_cache()  # 清除 CUDA 緩存

        # 📌 更新 epoch 總 loss
        epoch_loss += bce_loss.item()

        print(f"Epoch {epoch+1}, Iteration {iteration}, BCE Loss: {bce_loss.item()}")
        iteration += 1

    # 計算 epoch 平均 loss 並更新學習率
    epoch_loss /= len(train_loader)  # 計算 loss 平均值
    scheduler.step(epoch_loss)  # 使用 ReduceLROnPlateau 更新 lr

    print(f"Epoch {epoch+1} 結束，平均 BCE Loss: {epoch_loss}, 當前學習率: {optimizer.param_groups[0]['lr']}")
    epoch += 1

    
    # ===== Validation Step =====
    net.eval()  # 切換為評估模式
    val_loss = 0.0
    num_batches = 0
    num_samples = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_idx, val_graph_batch in enumerate(val_loader):  # 使用 enumerate 確保 batch 索引
            if num_samples >= num_val_samples:
                break  # 達到 50,000 筆後結束驗證
    
            val_batch = dotdict(
                graph = val_graph_batch.to(DEVICE),
                bind = torch.from_numpy(val_bind_subset[batch_idx * 9 : (batch_idx + 1) * 9]).to(DEVICE),
            )

    
            net.output_type = ['loss', 'infer']
            val_output = net(val_batch)
            val_loss += val_output['bce_loss'].item()
            num_batches += 1
            num_samples += len(val_batch.graph)
    
            # 取得預測值（通常為機率），轉為 0/1
            probs = val_output['bind'].detach().cpu().numpy()  # 轉為 NumPy
            preds = (probs > 0.5).astype(int)  # 設定閾值 0.5
            labels = val_batch.bind.cpu().numpy()
    
            all_preds.append(preds)
            all_labels.append(labels)
    
    # 計算平均 Loss
    avg_val_loss = val_loss / num_batches
    
    # 將所有 batch 的預測值與標籤合併
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # 計算 Accuracy 和 F1 Score
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')  # 'macro' 計算所有類別的平均 F1
    
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")

    epoch += 1


'\nimport numpy as np\nimport torch\nfrom torch_geometric.data import Data\nfrom multiprocessing import Pool\nfrom tqdm import tqdm\nfrom torch_geometric.loader import DataLoader as PyGDataLoader\nfrom sklearn.metrics import accuracy_score, f1_score\nfrom torch.optim.lr_scheduler import ReduceLROnPlateau\n\n# 轉換為 PyG 格式\ndef to_pyg_list(graph):\n\tL = len(graph)\n\tfor i in tqdm(range(L)):\n\t\tN, edge, node_feature, edge_feature = graph[i]\n\t\tgraph[i] = Data(\n\t\t\tidx=i,\n\t\t\tedge_index=torch.from_numpy(edge.T).int(),\n\t\t\tx=torch.from_numpy(node_feature).byte(),\n\t\t\tedge_attr=torch.from_numpy(edge_feature).byte(),\n\t\t)\n\treturn graph\n\n\ntrain_file = \'/kaggle/input/trainall7/sampled_train_all_7.parquet\'\ndf = pd.read_parquet(train_file)\n\n\n# train_loader\n# 從 df 讀取 SMILES\ntrain_smiles = df[\'molecule_smiles\'].tolist()\n\n\ntrain_bind = df[["BRD4", "HSA", "sEH"]].values\n#train_bind = np.array([[1, 0, 0] if b == 1 else [0, 0, 0] for b in train_bind])  # 假設 binds 是

In [25]:

model_path = "/kaggle/working/gnn_model_finish.pth"
torch.save(net, model_path)
print(f"模型已保存至 {model_path}")


'\nmodel_path = "/kaggle/working/gnn_model_finish.pth"\ntorch.save(net, model_path)\nprint(f"模型已保存至 {model_path}")\n'

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader as PyGDataLoader
from multiprocessing import Pool

# 設定設備 (GPU or CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入模型並切換為推理模式
net = torch.load("/kaggle/input/gnn-v9-9-2/pytorch/default/1/gnn_model_finish9.pth", map_location=DEVICE)
net.to(DEVICE)
net.eval()

In [ ]:
# 設定讀取 Parquet 文件
filename = '/kaggle/input/test-110/sampled_test_all.parquet'
df = pd.read_parquet(filename)

# 提取測試數據
test_smiles = df['molecule_smiles'].tolist()
y_true = df[["BRD4", "HSA", "sEH"]].values  # 轉為 NumPy (Shape: [N, 3])

num_test = len(test_smiles)

# 轉換 SMILES → Graph（使用 8 核心加速）
with Pool(processes=8) as pool:
    test_graph = list(tqdm(pool.imap(smile_to_graph, test_smiles), total=num_test))

# 轉換為 PyG 格式
test_graph = to_pyg_list(test_graph)

# 建立 PyG DataLoader
test_loader = PyGDataLoader(test_graph, batch_size=3, shuffle=False, drop_last=False, worker_init_fn=np.random.seed(42))


In [ ]:
import pandas as pd


y_true_df = pd.DataFrame(y_true)  
y_true_df = y_true_df.fillna(0.0)
y_true_df.isnull().sum()
y_true_df

In [ ]:
# ===== 進行推論 =====
all_preds = []
all_probs = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        batch = batch.to(DEVICE)  # 確保數據在正確設備上
        
        # 構造模型輸入
        test_batch = {"graph": batch}

        # 獲取模型輸出
        net.output_type = ['infer']  # 設置為推理模式
        test_output = net(test_batch)

        # 獲取預測機率
        probs = test_output["bind"].detach().cpu().numpy()  # Shape: [batch_size, 3]
        
        # 轉換為 0/1 預測值 (閾值 0.5)
        preds = (probs > 0.5).astype(int)

        # 儲存結果
        all_probs.append(probs)
        all_preds.append(preds)

# 合併所有 batch 的預測結果
all_probs = np.vstack(all_probs)  # 預測機率
all_preds = np.vstack(all_preds)  # 二元預測結果
all_labels = y_true_df.to_numpy()

# ===== 計算指標 =====
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')  # 'macro' 計算所有類別的平均 F1
auc = roc_auc_score(all_labels, all_probs, average='macro')  # 直接使用機率計算 AUC

# ===== 輸出結果 =====

print(f"Test F1 Score: {f1:.4f}")
print(f"Test AUC Score: {auc:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


submission

In [27]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader as PyGDataLoader
from multiprocessing import Pool

# 設定設備 (GPU or CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入模型並切換為推理模式
net = torch.load("/kaggle/input/gnn-v9-9-2/pytorch/default/1/gnn_model_finish9.pth", map_location=DEVICE)
net.to(DEVICE)
net.eval()

<ipython-input-27-507fd60f1ffb>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net = torch.load("/kaggle/input/gnn-v9-9-2/pytorch/default/1/gnn_model_finish9.pth", map_lo

Net(
  (smile_encoder): MPNNModel(
    (lin_in): Linear(in_features=72, out_features=96, bias=True)
    (convs): ModuleList(
      (0-3): 4 x MPNNLayer(emb_dim=96, aggr=add)
    )
  )
  (bind): Sequential(
    (0): Linear(in_features=96, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU(inplace=True)
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=512, out_features=3, bias=True)
  )
)

In [28]:
import duckdb
import pandas as pd
from tqdm import tqdm
import numpy as np # linear algebra

In [29]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission_gnn.csv'  # Specify the path and filename for the output file

test = pd.read_csv(test_file)
test.shape

(1674896, 6)

In [30]:
protein_mapping = {
    "BRD4": 0,
    "HSA": 1,
    "sEH": 2
}

test['protein_name'] = test['protein_name'].map(protein_mapping)

test

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,0
1,295246831,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,1
2,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,2
3,295246833,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,0
4,295246834,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,1
...,...,...,...,...,...,...
1674891,296921721,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...,1
1674892,296921722,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...,2
1674893,296921723,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2cccs2)nc(Nc2noc3...,0
1674894,296921724,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2cccs2)nc(Nc2noc3...,1


In [31]:
'''
# 提取測試數據
test_smiles = df['molecule_smiles'].tolist()


num_test = len(test_smiles)

# 轉換 SMILES → Graph（使用 8 核心加速）
with Pool(processes=8) as pool:
    test_graph = list(tqdm(pool.imap(smile_to_graph, test_smiles), total=num_test))

# 轉換為 PyG 格式
test_graph = to_pyg_list(test_graph)

# 建立 PyG DataLoader
test_loader = PyGDataLoader(test_graph, batch_size=16, shuffle=False, drop_last=False)

all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):

        batch = batch.to(DEVICE)  # 確保數據在正確設備上
                
        # 構造模型輸入
        test_batch = {"graph": batch}
        
        # 獲取模型輸出
        net.output_type = ['infer']  # 設置為推理模式
        test_output = net(test_batch)
        
        # 獲取預測機率
        probs = test_output["bind"].detach().cpu().numpy()  # Shape: [batch_size, 3]
        probs = pd.DataFrame(probs.values.flatten()
        
        # 轉換為 0/1 預測值 (閾值 0.5)
        preds = (probs > 0.5).astype(int)
        
        # 儲存結果
        all_preds.append(preds)

# Create a DataFrame with 'id' and 'probability' columns
output_df = pd.DataFrame({'id': test['id'], 'binds': all_preds})

# Save the output DataFrame to a CSV file
output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
'''

'\n# 提取測試數據\ntest_smiles = df[\'molecule_smiles\'].tolist()\n\n\nnum_test = len(test_smiles)\n\n# 轉換 SMILES → Graph（使用 8 核心加速）\nwith Pool(processes=8) as pool:\n    test_graph = list(tqdm(pool.imap(smile_to_graph, test_smiles), total=num_test))\n\n# 轉換為 PyG 格式\ntest_graph = to_pyg_list(test_graph)\n\n# 建立 PyG DataLoader\ntest_loader = PyGDataLoader(test_graph, batch_size=16, shuffle=False, drop_last=False)\n\nall_preds = []\nwith torch.no_grad():\n    for batch in tqdm(test_loader, desc="Testing"):\n\n        batch = batch.to(DEVICE)  # 確保數據在正確設備上\n                \n        # 構造模型輸入\n        test_batch = {"graph": batch}\n        \n        # 獲取模型輸出\n        net.output_type = [\'infer\']  # 設置為推理模式\n        test_output = net(test_batch)\n        \n        # 獲取預測機率\n        probs = test_output["bind"].detach().cpu().numpy()  # Shape: [batch_size, 3]\n        probs = pd.DataFrame(probs.values.flatten()\n        \n        # 轉換為 0/1 預測值 (閾值 0.5)\n        preds = (probs > 0.5).astype(int)\n 

In [32]:
# 轉換為 PyG 格式
def to_pyg_list(graph):
	L = len(graph)
	for i in tqdm(range(L)):
		N, edge, node_feature, edge_feature = graph[i]
		graph[i] = Data(
			idx=i,
			edge_index=torch.from_numpy(edge.T).int(),
			x=torch.from_numpy(node_feature).byte(),
			edge_attr=torch.from_numpy(edge_feature).byte(),
		)
	return graph

In [34]:
import torch
import pandas as pd
import os
from tqdm import tqdm
from multiprocessing import Pool

BATCH_SIZE = 3  # 減少 batch_size 來降低記憶體負擔
CHUNK_SIZE = 50000  # 每次處理 50000 筆
OUTPUT_FILE = "output.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# **確保 CSV 不會重複寫入 header**
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)  # 先刪除，避免重複 append

# **逐批處理 SMILES**
for chunk_start in range(0, len(test), CHUNK_SIZE):
    chunk_end = min(chunk_start + CHUNK_SIZE, len(test))
    test_chunk = test.iloc[chunk_start:chunk_end]  # **取出一小部分**
    
    test_smiles = test_chunk['molecule_smiles'].tolist()
    test_protein_names = test_chunk['protein_name'].tolist()
    test_ids = test_chunk['id'].tolist()
    
    # **逐批轉換 SMILES → Graph**
    with Pool(processes=8) as pool:
        test_graph = list(tqdm(pool.imap(smile_to_graph, test_smiles), total=len(test_smiles)))

    # **轉換為 PyG 格式**
    test_graph = to_pyg_list(test_graph)

    # **建立 DataLoader**
    test_loader = PyGDataLoader(test_graph, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

    all_preds = []  # **存放當前 chunk 的預測結果**
    batch_start = 0  # **批次索引追蹤**

    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f"Testing {chunk_start}/{len(test)}"):
            batch_size = batch.num_graphs
            batch = batch.to(DEVICE)

            # **構造模型輸入**
            test_batch = {"graph": batch}
            net.output_type = ['infer']
            test_output = net(test_batch)  # **輸出形狀 [batch_size, num_proteins]**

            for i in range(batch_size):  
                protein_name = test_protein_names[batch_start + i]

                if protein_name >= test_output["bind"].size(1):
                    print(f"Warning: protein_name {protein_name} out of bounds for batch {i}")
                    continue
                
                protein_pred = test_output["bind"][i, protein_name]
                all_preds.append(1 if protein_pred > 0.5 else 0)

            batch_start += batch_size

    # **寫入 CSV，減少記憶體佔用**
    output_df = pd.DataFrame({'id': test_ids, 'binds': all_preds})
    output_df.to_csv(OUTPUT_FILE, index=False, mode='a', header=not os.path.exists(OUTPUT_FILE))

    del test_graph, test_loader, all_preds  # **釋放記憶體**
    torch.cuda.empty_cache()  # **清理 GPU 記憶體**


Testing 1650000/1674896: 100%|██████████| 8299/8299 [00:45<00:00, 182.40it/s]


In [42]:
import pandas as pd

input_file = '/kaggle/working/output.csv'
output_file = '/kaggle/working/submission_gnn.csv'
# 讀取已儲存的 CSV
output_df = pd.read_csv(input_file)

print(len(output_df))

# 修改 id 欄位
output_df['id'] = range(295246830, 295246830 + len(output_df))

print(output_df.shape)

# 將修改後的 DataFrame 儲存回 CSV
output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))


1674896
(1674896, 2)
